In [104]:
import pandas as pd
data = pd.read_csv('age_of_marriage_data.csv')
print(data.shape)
data.head()

(2567, 10)


,id,gender,height,religion,caste,mother_tongue,profession,location,country,age_of_marriage
0,1,female,"5'4""",NaN,others,Telugu,NaN,London,United Kingdom,21.0
1,2,male,"5'7""",Jain,Shwetamber,Gujarati,Doctor / Healthcare Professional,Fairfax- VA,USA,32.0
2,3,male,"5'7""",Hindu,Brahmin,Hindi,Entrepreneurs / Business,Begusarai,India,32.0
3,4,female,"5'0""",Hindu,Thakur,Hindi,Architect,Mumbai,India,30.0
4,5,male,"5'5""",Christian,Born Again,Malayalam,Sales Professional / Marketing,Sulthan Bathery,India,30.0


In [105]:
data.dropna(axis=0,how='any',subset=['age_of_marriage','height'],inplace = True)
data.shape

(2430, 10)

In [106]:
data.isnull().sum()

id                   0
gender               8
height               0
religion           498
caste              105
mother_tongue      108
profession         255
location           101
country             16
age_of_marriage      0
dtype: int64

In [107]:
X = data.loc[:,['gender','height','religion','caste','mother_tongue','country','profession','location']]
#y = data.loc[:,['age_of_marriage']]
y = data.age_of_marriage

In [108]:
X.head()

,gender,height,religion,caste,mother_tongue,country,profession,location
0,female,"5'4""",NaN,others,Telugu,United Kingdom,NaN,London
1,male,"5'7""",Jain,Shwetamber,Gujarati,USA,Doctor / Healthcare Professional,Fairfax- VA
2,male,"5'7""",Hindu,Brahmin,Hindi,India,Entrepreneurs / Business,Begusarai
3,female,"5'0""",Hindu,Thakur,Hindi,India,Architect,Mumbai
4,male,"5'5""",Christian,Born Again,Malayalam,India,Sales Professional / Marketing,Sulthan Bathery


In [109]:
X.shape

(2430, 8)

In [110]:
X.isnull().sum()

gender             8
height             0
religion         498
caste            105
mother_tongue    108
country           16
profession       255
location         101
dtype: int64

In [111]:
def h_cms(h):
    return int(h.split('\'')[0])*30.48 + int(h.split('\'')[1].replace('"',''))*2.54

X['height_cms'] = X.height.apply(h_cms)

X.drop('height',inplace=True,axis=1)
X.head()

,gender,religion,caste,mother_tongue,country,profession,location,height_cms
0,female,NaN,others,Telugu,United Kingdom,NaN,London,162.56
1,male,Jain,Shwetamber,Gujarati,USA,Doctor / Healthcare Professional,Fairfax- VA,170.18
2,male,Hindu,Brahmin,Hindi,India,Entrepreneurs / Business,Begusarai,170.18
3,female,Hindu,Thakur,Hindi,India,Architect,Mumbai,152.40
4,male,Christian,Born Again,Malayalam,India,Sales Professional / Marketing,Sulthan Bathery,165.10


In [112]:
X.fillna('#', inplace=True) 

In [113]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
X.loc[:,['gender','religion','caste','mother_tongue','country','profession','location']]= \
X.loc[:,['gender','religion','caste','mother_tongue','country','profession','location']].apply(enc.fit_transform)

In [114]:
X.head()

,gender,religion,caste,mother_tongue,country,profession,location,height_cms
0,1,0,42,25,27,0,294,162.56
1,2,3,34,7,25,23,167,170.18
2,2,2,14,9,7,26,61,170.18
3,1,2,36,9,7,8,330,152.40
4,2,1,13,14,7,71,460,165.10


In [116]:
import numpy as np
X = X.replace(to_replace = 0, value = np.nan)
X.head()

,gender,religion,caste,mother_tongue,country,profession,location,height_cms
0,1.0,NaN,42.0,25.0,27.0,NaN,294.0,162.56
1,2.0,3.0,34.0,7.0,25.0,23.0,167.0,170.18
2,2.0,2.0,14.0,9.0,7.0,26.0,61.0,170.18
3,1.0,2.0,36.0,9.0,7.0,8.0,330.0,152.40
4,2.0,1.0,13.0,14.0,7.0,71.0,460.0,165.10


In [118]:
#KNN imputer
from sklearn.impute import KNNImputer
import pandas as pd

imputer = KNNImputer()
imputed_data = imputer.fit_transform(X)  # impute all the missing data
df_temp = pd.DataFrame(imputed_data)
df_temp.columns = X.columns

In [120]:
X = df_temp

In [124]:
X.head()

,gender,religion,caste,mother_tongue,country,profession,location,height_cms
0,1.0,2.6,42.0,25.0,27.0,58.6,294.0,162.56
1,2.0,3.0,34.0,7.0,25.0,23.0,167.0,170.18
2,2.0,2.0,14.0,9.0,7.0,26.0,61.0,170.18
3,1.0,2.0,36.0,9.0,7.0,8.0,330.0,152.40
4,2.0,1.0,13.0,14.0,7.0,71.0,460.0,165.10


In [131]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [136]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=80,max_depth=11)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)

In [137]:
from sklearn.metrics import mean_absolute_error, r2_score
print("MAE : ", mean_absolute_error(y_test,y_predict))
r2_score(y_test,y_predict)

MAE :  1.4847909135902344


0.45921555353008814